In [110]:
import textract
import spacy
nlp = spacy.load('en')

In [111]:
text = textract.process("Problem Assignment/ordjud.docx")
text1 = textract.process("Problem Assignment/ordjud1.docx")
text2 = textract.process("Problem Assignment/ordjud2.docx")

In [171]:
t = str(text,'utf-8')
t1 = str(text1,'utf-8')
t2 = str(text2,'utf-8')

In [173]:
import re
def preprocess(t):
    return re.sub(r'([\n \t])\1+', r'\1', t).replace('\t','')
new_t = preprocess(t)
new_t1 = preprocess(t1)
new_t2 = preprocess(t2)

In [174]:
data1 = new_t.split('\n')
data2 = new_t1.split('\n')
data3 = new_t2.split('\n')
len(data1),len(data2),len(data3)

(280, 414, 82)

In [134]:
def get_courtName(data):
    cnt = 0
    courtname = ""
    for i in data:
        cnt+=1
        if cnt <= 10:
            j = str(i).lower()
            if re.search('high court',j):
                #print(nlp(i).ents[0])
                courtname = nlp(i).ents[0]
    return courtname
courtname = get_courtName(data2)


In [183]:
def getapp_def_Name(data):
    applicant = ""
    defendent = ""
    cnt = 0
    for i in range(0,len(data)):
        cnt+=1
        if cnt <= 100:
            j = str(data[i]).lower()
            if re.search('vs',j) or re.search('versus',j):
                if applicant == "" and defendent =="":
                    applicant,defendent = data[i-2]+data[i-1],data[i+1]+data[i+2]
                #print(applicant)
                #print(defendent)
    return applicant,defendent
getapp_def_Name(data1)
getapp_def_Name(data3)
                    

('Navin S/o. Mahaveer Prasad Sarawagi& Ors.…Petitioners',
 'Pradhana @ Pinki W/o. Rajesh Singhania…RespondentMr. Udaipuri, Adv. a/w. Mr. J P Dhanuka, Adv. for petitioners.')

In [117]:
data1

['KPPNair1NMSL 1717 of 2013',
 'IN THE HIGH COURT OF JUDICATURE AT BOMBAY',
 'ORDINARY ORIGINAL CIVIL JURISDICTION',
 'NOTICE OF MOTION (L) NO. 1717 OF 2013',
 'IN',
 'SUIT (L) NO. 752 OF 2013',
 'Pidilite Industries Limited',
 '...Applicant',
 'In the matter between:',
 'Pidilite Industries Limited',
 '...Plaintiff',
 'vs.',
 'Jubilant Agri & Consumer Products Limited',
 '...Defendant',
 'Mr. V.R. Dhond, Senior Advocate, along with Mr. Nishad Nadkarni, Mr. Vaibhav Keni and Mr. Sumeet Rane, instructed by M/s. LEGASIS PARTNERS, for the Applicant/Plaintiff.',
 'Mr. Ravi Kadam, Senior Advocate, along with Mr. A.A. Kirpekar, instructed by M/s.',
 'MAG LEGAL, for the Defendant.',
 'CORAM:S.J. KATHAWALLA, J.',
 'Order reserved on:10th October, 2013',
 'Order pronounced on: 13th January, 2014',
 'ORDER:',
 'The above Suit is filed by the Plaintiff against the Defendant on the ground that the Defendant is infringing the registered trademarks of the Plaintiff, infringing the copyrights of the P

In [132]:
def get_judge(data):
    judge = ""
    for i in data:
        cnt = 0
        if cnt<= 100:
            j = str(i).lower()
            if re.search('coram',j):
                befor_keyowrd, keyword, after_keyword = j.partition(':')
                #print(after_keyword)
                judge = after_keyword
    return judge
get_judge(data1)

's.j. kathawalla, j.'

In [160]:
data1

['KPPNair1NMSL 1717 of 2013',
 'IN THE HIGH COURT OF JUDICATURE AT BOMBAY',
 'ORDINARY ORIGINAL CIVIL JURISDICTION',
 'NOTICE OF MOTION (L) NO. 1717 OF 2013',
 'IN',
 'SUIT (L) NO. 752 OF 2013',
 'Pidilite Industries Limited',
 '...Applicant',
 'In the matter between:',
 'Pidilite Industries Limited',
 '...Plaintiff',
 'vs.',
 'Jubilant Agri & Consumer Products Limited',
 '...Defendant',
 'Mr. V.R. Dhond, Senior Advocate, along with Mr. Nishad Nadkarni, Mr. Vaibhav Keni and Mr. Sumeet Rane, instructed by M/s. LEGASIS PARTNERS, for the Applicant/Plaintiff.',
 'Mr. Ravi Kadam, Senior Advocate, along with Mr. A.A. Kirpekar, instructed by M/s.',
 'MAG LEGAL, for the Defendant.',
 'CORAM:S.J. KATHAWALLA, J.',
 'Order reserved on:10th October, 2013',
 'Order pronounced on: 13th January, 2014',
 'ORDER:',
 'The above Suit is filed by the Plaintiff against the Defendant on the ground that the Defendant is infringing the registered trademarks of the Plaintiff, infringing the copyrights of the P

In [168]:
import pdb
def get_dates(data):
    cnt = 0
    reserved_date = ""
    pronounced_date = ""
    for i in data:
        if cnt <= 100:
            j = str(i).lower()
            if re.search('reserving',j) or re.search('reserve',j):
                befor_keyowrd, keyword, after_keyword = j.partition(':')
                #print(j.split(":")[1])
                #print(after_keyword)
                if after_keyword!='':
                    reserved_date = after_keyword
            if re.search('pronounc',j):
                befor_keyowrd, keyword, after_keyword = j.partition(':')
                #print(after_keyword)
                #print(j.split(":")[1])
                if after_keyword!="":
                    pronounced_date = after_keyword
        #pdb.set_trace()
    #print(reserved_date)
    return reserved_date,pronounced_date
get_dates(data1)

('10th october, 2013', ' 13th january, 2014')

In [177]:
#judgement group
citation_title = ""
citation = ""
Act_title = ""
secref_title = ""
sec_obj = []
act_obj = []
def get_act_sec(data):
    cnt = 0
    x,y = [],[]
    for i in data:
        j = str(i).lower()
        #print(i)
        #sec = re.search('[^\n][.]\w[a-z]\act,\s\d{4}',i)
        sec = re.search(r'Section \d+',i)
        sec_act = re.search(r'\w+ \w+ \w+ Act, \d+',i)
        cnt+=1
        #print(cnt)
        if sec!=None:
            #print(sec.group())
            x.append(sec.group())
        if sec_act != None:
            y.append(sec_act.group())
            #print(sec_act.group())
    return x,y


In [93]:
data1

['KPPNair1NMSL 1717 of 2013',
 'IN THE HIGH COURT OF JUDICATURE AT BOMBAY',
 'ORDINARY ORIGINAL CIVIL JURISDICTION',
 'NOTICE OF MOTION (L) NO. 1717 OF 2013',
 'IN',
 'SUIT (L) NO. 752 OF 2013',
 'Pidilite Industries Limited',
 '...Applicant',
 'In the matter between:',
 'Pidilite Industries Limited',
 '...Plaintiff',
 'vs.',
 'Jubilant Agri & Consumer Products Limited',
 '...Defendant',
 'Mr. V.R. Dhond, Senior Advocate, along with Mr. Nishad Nadkarni, Mr. Vaibhav Keni and Mr. Sumeet Rane, instructed by M/s. LEGASIS PARTNERS, for the Applicant/Plaintiff.',
 'Mr. Ravi Kadam, Senior Advocate, along with Mr. A.A. Kirpekar, instructed by M/s.',
 'MAG LEGAL, for the Defendant.',
 'CORAM:S.J. KATHAWALLA, J.',
 'Order reserved on:10th October, 2013',
 'Order pronounced on: 13th January, 2014',
 'ORDER:',
 'The above Suit is filed by the Plaintiff against the Defendant on the ground that the Defendant is infringing the registered trademarks of the Plaintiff, infringing the copyrights of the P

In [51]:
def merge_paragraphs(data):
    par = ""
    dup = []
    cnt = 0
    flag = -999 #0 for an index 1 for a text.
    for i in data:
        cnt+=1
        if re.match('^\d+\.*$',i):
            if (int(i.split('.')[0])) == cnt:
                flag=0
                par = ""
                par += i
        else:
            if flag==1:
                par=par+1
        if par!="":
            dup.append(par)
    return dup
dup1 = merge_paragraphs(data1)


In [184]:
#ordjud details:
reserved_dates,pronounced_dates = get_dates(data1)
app,defen = getapp_def_Name(data1)
sec_obj,act_obj = get_act_sec(data1)
print("court_name: ",get_courtName(data1))
print("applicant name: ",app)
print("defendent name: ",defen)
print("judge name: ",get_judge(data1))
print("reserved date: ",reserved_dates)
print("pronounced date: ",pronounced_dates)
print("sections: ",set(sec_obj))
print("act names: ",set(act_obj))

court_name:  BOMBAY
applicant name:  Pidilite Industries Limited...Plaintiff
defendent name:  Jubilant Agri & Consumer Products Limited...Defendant
judge name:  s.j. kathawalla, j.
reserved date:  10th october, 2013
pronounced date:   13th january, 2014
sections:  {'Section 16', 'Section 30', 'Section 17', 'Section 35', 'Section 9'}
act names:  {'the Trade Marks Act, 1999'}


In [185]:
#ordjud1 details
reserved_dates,pronounced_dates = get_dates(data2)
app,defen = getapp_def_Name(data2)
sec_obj,act_obj = get_act_sec(data2)
print("court_name: ",get_courtName(data2))
print("applicant name: ",app)
print("defendent name: ",defen)
print("judge name: ",get_judge(data2))
print("reserved date: ",reserved_dates)
print("pronounced date: ",pronounced_dates)
print("sections: ",set(sec_obj))
print("act names: ",set(act_obj))

court_name:  BOMBAY
applicant name:  Parbhani. Dist.Parbhani....Petitioner..
defendent name:  1]The State of Maharashtra,through its Principal Secretary,
judge name:   b.p. dharmadhikari
reserved date:   03.12.2014
pronounced date:   12.12.2014
sections:  {'Section 6'}
act names:  {'the Essential Commodities Act, 1955'}


In [186]:
#ordjud2 details as we can see the results are not that great for ordjud2 because the data are 
#not formatted in the same way as the earlier 2 documents, hence additional info for defendent
reserved_dates,pronounced_dates = get_dates(data3)
app,defen = getapp_def_Name(data3)
sec_obj,act_obj = get_act_sec(data3)
print("court_name: ",get_courtName(data3))
print("applicant name: ",app)
print("defendent name: ",defen)
print("judge name: ",get_judge(data3))
print("reserved date: ",reserved_dates)
print("pronounced date: ",pronounced_dates)
print("sections: ",set(sec_obj))
print("act names: ",set(act_obj))

court_name:  BOMBAY
applicant name:  Navin S/o. Mahaveer Prasad Sarawagi& Ors.…Petitioners
defendent name:  Pradhana @ Pinki W/o. Rajesh Singhania…RespondentMr. Udaipuri, Adv. a/w. Mr. J P Dhanuka, Adv. for petitioners.
judge name:  mrs. roshan dalvi, j.
reserved date:  
pronounced date:  
sections:  set()
act names:  {'Guardian and Wards Act, 1890'}
